<a href="https://colab.research.google.com/github/jwkanggist/EverybodyTensorflow2.0/blob/master/lab18_mobilenet_cifar100_tf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LAB18: Mobilenet for Cifar100

Depthwise-separable *convolution을* 구현하여 moblienet을 만들어 보자
- Cifar100 datatset을 위한 실험


In [0]:
# preprocessor parts

from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard

import numpy as np
import pandas as pd

from matplotlib import cm
from matplotlib import gridspec
import matplotlib.pyplot as plt

from datetime import datetime

In [0]:
  # for Tensorboard use
  LOG_DIR = 'drive/data/tb_logs'
	
  !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
  !unzip ngrok-stable-linux-amd64.zip
	
  import os
  if not os.path.exists(LOG_DIR):
    os.makedirs(LOG_DIR)
	  
  get_ipython().system_raw(
      'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
      .format(LOG_DIR))
	
  get_ipython().system_raw('./ngrok http 6006 &')
	
  !curl -s http://localhost:4040/api/tunnels | python3 -c \
      "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
# dataset loading part
# 데이터 파이프라인 부분 

cifar100 = tf.keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

print("x_train.shape = %s" % str(x_train.shape))
print("y_train.shape = %s" % str(x_train.shape))

x_train = x_train.reshape([x_train.shape[0],
                           x_train.shape[1],
                           x_train.shape[2],3])

x_test = x_test.reshape([x_test.shape[0],
                           x_test.shape[1],
                           x_test.shape[2],3])


x_train = np.array(tf.keras.backend.resize_images(x_train, height_factor=2,width_factor=2, data_format='channels_last',interpolation='bilinear'))
x_test = np.array(tf.keras.backend.resize_images(x_test, height_factor=2,width_factor=2, data_format='channels_last',interpolation='bilinear'))

print("x_train.shape = %s" % str(x_train.shape))
print("x_test.shape = %s" % str(x_test.shape))

print("y_train.shape = %s" % str(y_train.shape))
print("y_test.shape = %s" % str(y_test.shape))

input_shape = x_train.shape[1]

In [0]:
# Network modules
def conv_batchnorm_relu(net_in,filters,kernel_size,strides):
  net = tf.keras.layers.Conv2D(filters,kernel_size=(kernel_size,kernel_size),
                               strides=(strides,strides),
                               padding='same',
                               kernel_initializer='he_normal',
                               activation=None)(net_in)
  net = tf.keras.layers.BatchNormalization()(net)
  net = tf.keras.layers.Activation(activation='relu')(net)
  return net


def convdw_batchnorm_relu(net_in,filters,depth_multiplier,kernel_size,strides):
  net = tf.keras.layers.DepthwiseConv2D(kernel_size=(kernel_size,kernel_size),
                                        strides=(strides,strides),
                                        depth_multiplier=depth_multiplier,
                                        padding='same',
                                        kernel_initializer='he_normal',
                                        activation=None)(net_in)
  net = tf.keras.layers.BatchNormalization()(net)
  net = conv_batchnorm_relu(net,filters,1,1)

  return net


depth_multiplier=1
# moblienet model
net_in = tf.keras.layers.Input(shape=(input_shape,input_shape,3))
net = tf.keras.layers.Conv2D(64,kernel_size=(9,9),strides=1,padding='valid', kernel_initializer='he_normal',activation='relu')(net_in)
# net = convdw_batchnorm_relu(net,32,3,1)
# net = convdw_batchnorm_relu(net,64,3,2)
net = convdw_batchnorm_relu(net,128,depth_multiplier,3,1)
net = convdw_batchnorm_relu(net,128,depth_multiplier,3,2)
net = convdw_batchnorm_relu(net,256,depth_multiplier,3,1)
net = convdw_batchnorm_relu(net,256,depth_multiplier,3,2)
net = convdw_batchnorm_relu(net,512,depth_multiplier,3,1)
net = convdw_batchnorm_relu(net,512,depth_multiplier,3,1)
net = convdw_batchnorm_relu(net,512,depth_multiplier,3,1)
net = convdw_batchnorm_relu(net,512,depth_multiplier,3,1)
net = convdw_batchnorm_relu(net,512,depth_multiplier,3,1)
net = convdw_batchnorm_relu(net,512,depth_multiplier,3,2)
net = convdw_batchnorm_relu(net,1024,depth_multiplier,3,2)
net = tf.keras.layers.GlobalAvgPool2D(data_format='channels_last')(net)
net = tf.keras.layers.Dropout(0.4)(net)
net = tf.keras.layers.Dense(units=1000,activation='softmax')(net)
model = tf.keras.models.Model(inputs=net_in,outputs=net)


opt_fn = tf.keras.optimizers.Adam(learning_rate=1e-4,
                                beta_1=0.9,
                                beta_2=0.999)

# 'sparse_categorical_crossentropy' is for integer labels
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

tensorboard_callback = TensorBoard(log_dir=LOG_DIR, 
                           histogram_freq=1,
                           write_graph=True,
                           write_images=True)

In [0]:
# model training and evaluation part
training_epochs = 20
batch_size = 128
model.summary()
model.fit(x_train, y_train, 
          epochs=training_epochs,
          validation_data=(x_test, y_test),
          batch_size=batch_size,
          callbacks=[tensorboard_callback])
model.evaluate(x_test,  y_test, verbose=2)

In [0]:
# prediction 

test_input = x_test[300,:,:,:]
pred_y = model.predict(test_input.reshape([1,64,64,3]))

plt.figure(1)
plt.imshow(test_input.reshape([64,64,3]))
plt.title("input")
print("model prediction = %s"% pred_y.argmax())
